In [3]:
import os
os.getcwd()

'/Users/dklaupaa/Desktop/chat_wrap_package'

# 1. Generate Synthetic Framework 

In [3]:
from mypythonpackage import FrameworkGenerator
import pandas as pd

this is just a test for my self to see what functions i'd actually like for it to have - realised that it is wayyy too specific, needs to be more modular still. ++++
needs to be flexible in the input it takes in like categories should could be defined in this notebook :)

In [4]:

generator = FrameworkGenerator(model_name="llama-3.2-3b-instruct")


In [5]:
# Step 1 - Generate a framework
df = generator.generate_framework(
    prompt_path="src/framework_generation/outline_prompts/prompt_default.py", ### THIS NEEDS TO BE UPDATED TO ALSO TAKE IN A DIRECTORY DIRECTLY 
    num_samples=10,
    api_url="http://localhost:1234/v1/completions"
    #json_out="output/data.json",
    #csv_out="output/data.csv"
)

print("Snippet of the generated framework:")
print(df.head())

Generating for category: Clarification
Generating for category: Small Talk
Generating for category: Question
Number of duplicates removed: 5
Snippet of the generated framework:
                                                text       category
0               Could you explain the concept again?  Clarification
1  Would it help to provide an example or illustr...  Clarification
2  Can you explain the concept of "design thinkin...  Clarification
3  Can you rephrase the question to make it more ...  Clarification
4  Let me try to explain it again. The term "dist...  Clarification


In [9]:
# Step 2: Train + filter
# Either from CSV
filtered_df = generator.filter_with_classifier(
    train_data="data/tiny_labeled_data.csv",
    synth_data="data/generated_tuning_data/FINAL_synthetic_combined500.csv",
    filtered_save_path="data/generated_tuning_data/PACKAKGE_QUALDATA.csv"
)


Max token: 34 Average token: 15.186046511627907


Map: 100%|██████████| 11/11 [00:00<00:00, 5575.51 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/dklaupaa/Library/Caches/pypoetry/virtualenvs/chat-wrap-package-h94m8WQz-py3.12/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.928502,0.545455,0.797980,0.545455,0.496503
2,No log,0.683839,1.000000,1.000000,1.000000,1.000000
3,No log,0.537540,1.000000,1.000000,1.000000,1.000000
4,No log,0.472429,1.000000,1.000000,1.000000,1.000000


Map: 100%|██████████| 850/850 [00:00<00:00, 14122.91 examples/s]


Filtered data saved to data/generated_tuning_data/PACKAKGE_QUALDATA.csv


In [10]:
# Or entirely in memory
df_train = "data/tiny_labeled_data.csv"
df_synth = df

filtered_df = generator.filter_with_classifier(
    train_data=df_train,
    synth_data=df_synth
)

Max token: 34 Average token: 15.186046511627907


Map: 100%|██████████| 11/11 [00:00<00:00, 5172.35 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/dklaupaa/Library/Caches/pypoetry/virtualenvs/chat-wrap-package-h94m8WQz-py3.12/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.928502,0.545455,0.797980,0.545455,0.496503
2,No log,0.683839,1.000000,1.000000,1.000000,1.000000
3,No log,0.537540,1.000000,1.000000,1.000000,1.000000
4,No log,0.472429,1.000000,1.000000,1.000000,1.000000


Map: 100%|██████████| 25/25 [00:00<00:00, 6720.35 examples/s]


# 2. Generate Synthetic Tutor/Student Dialogue 
this is only for pof and probably not a part of the final pipeline 